In [ ]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Neural network libraries
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import EarlyStopping

# Reading images and creating video libraries
import cv2
from IPython.display import HTML
from base64 import b64encode
import matplotlib.animation as animation
import os

import SimpleITK as sitk
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
target = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
preds = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')

In [ ]:
# specify your image path
views = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
def load_imgs(idx, ignore_zeros=True, train=True):
    imgs = {}
    for view in views:
        save_ds = []
        if train:
            dir_path = os.walk(os.path.join(
            '../input/rsna-miccai-png/train/', idx, view
        ))
        else:
            dir_path = os.walk(os.path.join(
            '../input/rsna-miccai-png/test/', idx, view
        ))
        for path, subdirs, files in dir_path:
            for name in files:
                image_path = os.path.join(path, name) 
                ds = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                save_ds.append(np.array(ds))
        if len(save_ds) == 0:
            save_ds = np.zeros((1,256,256))
        imgs[view] = np.array(save_ds)
    return imgs

In [ ]:
from sklearn.preprocessing import StandardScaler

class DataGenerator3D(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels=None, batch_size=256, dim=(512,512,512), n_channels=3,
                 n_classes=2, shuffle=True, is_train=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.is_train = (labels is not None)
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        list_IDs_temp = self.list_IDs[index*self.batch_size:(index+1)*self.batch_size]

        X = self.__data_generation(list_IDs_temp)
        # Generate data
        if self.is_train:
            y = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            return np.array(X), np.array(y)
        else:
            return np.array(X)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            idx = str(ID).zfill(5)
            imgs = load_imgs(idx, ignore_zeros=False, train=self.is_train)
            new_imgs = []
            for ii in range(2):
                for jj in range(2):
                    if (ii == 1 and jj == 1):
                        continue
                    img_ = imgs[views[2*ii+jj]]
                    img_ = np.array([cv2.resize(img_[k], dsize=(self.dim[1],self.dim[0]), interpolation=cv2.INTER_LINEAR) for k in range(img_.shape[0])])
                    img_ = np.array([cv2.resize(img_.transpose(1,2,0)[k], dsize=(self.dim[2],self.dim[1]), interpolation=cv2.INTER_LINEAR) for k in range(self.dim[0])])
                    
                    # Removing radiofrequency inhomogeneity using N4 Bias Field Correction 
                    for p in range(len(img_)):
                        inputImage = sitk.GetImageFromArray(img_[p])
                        maskImage = sitk.GetImageFromArray((img_[p] >0.1) * 1)
                        inputImage = sitk.Cast(inputImage, sitk.sitkFloat32)
                        maskImage = sitk.Cast(maskImage, sitk.sitkUInt8)
                        corrector = sitk.N4BiasFieldCorrectionImageFilter()
                        numberFittingLevels = 4
                        maxIter = 100
                        if maxIter is not None:
                            corrector.SetMaximumNumberOfIterations([maxIter]
                                                                   * numberFittingLevels)
                        corrected_image = corrector.Execute(inputImage, maskImage)
                        img_[p] = sitk.GetArrayFromImage(corrected_image)
                        
                    # Normalization
                    sc = StandardScaler()
                    img_ = np.array([sc.fit_transform(img_[i]) for i in range(img_.shape[0])])

                    new_imgs.append(img_)
            new_imgs = np.concatenate(new_imgs).transpose(1,2,0).reshape((*self.dim,-1))
            X[i,] = new_imgs
        
        return X

In [ ]:
dim = (64,64,34)
N = (len(preds) // 5) * 5
test_dataset = DataGenerator3D(preds.BraTS21ID[:N], batch_size=5, dim=dim)
with tf.device('/gpu:0'):

    checkpoint_path = "../input/models/MIL_NN"
    model = tf.keras.models.load_model(checkpoint_path)
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC()]
        )
    
    preds_ = model.predict(test_dataset)

In [ ]:
preds.MGMT_value[:N] = preds_
preds.to_csv('submission.csv', index=False)